# Imports 

In [1]:
from common import getOrCreateSparkSession

In [2]:
spark = getOrCreateSparkSession()

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.2_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.2_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-83d5ad77-4d4b-431a-b673-809c9c329df8;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.2_2.12;1.4.2 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.2_2.12;0.74.0 in central
:: resolution report :: resolve 105ms :: artifacts dl 6ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.2_2.12;1.4.2 from central in [default]
	org.projectnessie.nessie-integrations#nessie-spark-extensions-3.2_2.12;0.74.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| sear

In [3]:
spark

# Clean Up

In [4]:
spark.sql("DROP TABLE IF EXISTS nessie.taxis.trips").toPandas()

""


In [5]:
spark.sql("DROP TABLE IF EXISTS nessie.taxis.new_trips").toPandas()

""


In [6]:
spark.sql("DROP NAMESPACE IF EXISTS nessie.taxis;").toPandas()

""


In [7]:
spark.sql("DROP BRANCH IF EXISTS batch2 IN nessie").toPandas()

,status
0,OK


# Prepare namespace and tables

In [8]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS nessie.taxis;").toPandas()

""


In [9]:
spark.sql("LIST REFERENCES IN nessie").toPandas()

,refType,name,hash
0,Branch,dev,aabe970e6cd4bd7c323d5fc66908da5baaab44063cbdb8...
1,Branch,main,b943563d86f2d0c8046d0ff54ffce1d6deb2aed42d778f...


In [10]:
spark.sql(
"""
CREATE TABLE IF NOT EXISTS nessie.taxis.trips(
    vendor_id INT,
    pickup_datetime TIMESTAMP,
    dropoff_datetime TIMESTAMP,
    passenger_count INT,
    pickup_location_id INT,
    dropoff_location_id INT,
    fare_amount FLOAT
)
USING iceberg;
"""
).toPandas()

""


# Ingest data into `@main`

In [11]:
spark.sql(
"""
CREATE OR REPLACE TEMPORARY VIEW trips_table USING csv
OPTIONS (path "/data/yellow_tripdata_sample_2019_01.csv", header true);
"""
).toPandas()

""


In [12]:
spark.sql("SELECT COUNT(*) FROM trips_table;").toPandas()

,count(1)
0,15


In [13]:
spark.sql("""
INSERT INTO nessie.taxis.trips
SELECT 
    CAST(vendor_id AS INT),
    CAST(pickup_datetime AS TIMESTAMP),
    CAST(dropoff_datetime AS TIMESTAMP),
    CAST(passenger_count AS INT),
    CAST(pickup_location_id AS INT),
    CAST(dropoff_location_id AS INT),
    CAST(fare_amount AS FLOAT)
FROM trips_table;
"""
).toPandas()

""


In [14]:
spark.sql("SELECT * FROM nessie.taxis.trips LIMIT 5;").toPandas()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_location_id,dropoff_location_id,fare_amount
0,1,2019-01-15 03:36:12,2019-01-15 03:42:19,1,230,48,6.5
1,1,2019-01-25 18:20:32,2019-01-25 18:26:55,1,112,112,6.0
2,1,2019-01-05 06:47:31,2019-01-05 06:52:19,1,107,4,6.0
3,1,2019-01-09 15:08:02,2019-01-09 15:20:17,1,143,158,11.0
4,1,2019-01-25 18:49:51,2019-01-25 18:56:44,1,246,90,6.5


In [15]:
spark.sql("SELECT COUNT(*) FROM nessie.taxis.trips;").toPandas()

,count(1)
0,15


Drop temporary view

In [16]:
spark.sql("DROP VIEW IF EXISTS trips_table").toPandas()

""


In [17]:
spark.sql("SHOW TABLES IN nessie;").toPandas()

,namespace,tableName,isTemporary
0,taxis,trips,False
